## 使用selenium模拟点击爬取球队球员信息

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import requests
from bs4 import BeautifulSoup

### 爬取球队信息

In [2]:
def parse_team_html(html, url):
    team_infos = {"url": url}
    soup = BeautifulSoup(html, 'lxml')
    _layout = soup.find(name="div", id="__layout")
    team_data_wrap = _layout.find(name="div", class_="team-data-wrap")
    team_con = team_data_wrap.find(name="div", class_="team-con")
    team_left = team_con.find(name="div", class_="team-left")
    team_right = team_con.find(name="div", class_="team-right")
    team_info = team_left.find(name="div", class_="team-info")
    team_relate_info = team_left.find(name="div", class_="team-relate-info")
    team_info_con = team_info.find(name="div", class_="info-con")

    #解析基本信息
    team_base_info = {}
    for spn in team_info_con.find_all(name="span"):
        # print(spn)
        con = spn.contents
        if len(con) == 2:
            team_base_info[con[0].string.strip("：").replace(" ", "")] = con[1]
        else:
            team_base_info[con[0].string.strip("：").replace(" ", "")] = ""
    team_name = team_info_con.find(name="p", class_="team-name").contents[0]
    team_base_info["team_name"] = team_name
    print(team_name)
    english_name = team_info_con.find(name="p", class_="en-name").contents[0]
    team_base_info["english_name"] = english_name
    address = team_info_con.find(name="p", class_="address").contents#解析地址
    if len(address) == 2:
        team_base_info[address[0].string.strip("：").replace(" ", "")] = address[1]
    else:
        team_base_info[address[0].string.strip("：").replace(" ", "")] = ""
    team_infos["team_base_info"] = team_base_info

    #解析球队的运动员
    team_player = []
    item_th = []
    for item in team_relate_info.find(name="p", class_="item-th").find_all(name="span"):
        # print(item)
        item_th.append(item.string)
    team_player.append(item_th)
    team_related_player = team_relate_info.find(name="div", class_="team-player-data")
    for item in team_related_player.find_all(name="p"):
    #     print(item)
        player_info = []
    #     item1 = item.find(name="span", class_="item1")
    #     player_info.append(item1.string)
    #     item2 = item.find(name="span", class_="item2")
    #     player_info.append(item2.string)
    #     item3 = item.find(name="span", class_="item3")
    #     player_info.append(item3.)
        for it in item.find_all(name="span"):
    #         print(it)
            player_info.append(it.find(text=True))
        team_player.append(player_info)
    team_infos["team_related_player"] = team_player

    #解析球队荣誉数据
    hornor = team_relate_info.find(name="div", class_="hornor-record")
    hornor_dict = {}
    for item in hornor.find_all(name="div", class_="hornor-list"):
    #     print(item.prettify())
        champion = item.find(name="span", class_="champion", text=True)
        time = item.find_all(name="b")
        hornor_dict[champion.string.split("X")[0].strip()] = [e.string for e in time]
    team_infos["honor_record"] = hornor_dict

    return team_infos

In [3]:
def get_teams_data(url):
    browser = webdriver.Chrome()   
    try:
        browser.get(url)
        
#         返回元素或者异常NoSuchElementException（当不存在该元素时）
        click = browser.find_element_by_class_name("team_point_ranking")
        teams = click.find_elements_by_class_name("td")
        
        for i, team in enumerate(teams):
            team.click()
    #         print(browser.window_handles)
            browser.switch_to.window(browser.window_handles[1])#切换到跳转后的页面
            teamsUrl = browser.current_url
            print(teamsUrl)#获取该页面url
            with open("teamsUrl.txt", "a+", encoding="utf-8") as urlf:
                urlf.write(teamsUrl + "\n")
            #把富力排除，因为页面结构不一样
            if not browser.current_url.startswith("https://www.dongqiudi.com/team/"):   
                pass
            else:
    #             print(parse_team_html(browser.page_source, browser.current_url))
                team_infos = parse_team_html(browser.page_source, browser.current_url)
                with open("teamsData.txt", "a+", encoding="utf-8") as f:
                        info = "{}\n".format(team_infos)
                        f.write(info)
            browser.close()
            browser.switch_to.window(browser.window_handles[0])#切换回原页面
    #         print(browser.window_handles)
    #         print(browser.get_cookies())
    #         print(team.page_source)
    #         print(parse_team_html(team.page_source))
    
    except NoSuchElementException:
        print("***************************\n      非有效页面          \n***********************")
    finally:
        browser.quit()

In [27]:
base_url= "https://www.dongqiudi.com/data/"
for i in range(1, 500):
    url = base_url + str(i)
    print(url)
    get_teams_data(url)    

https://www.dongqiudi.com/data/50
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/51
https://www.dongqiudi.com/data/52
https://www.dongqiudi.com/data/53
https://www.dongqiudi.com/team/50000666.html
赫尔辛基
https://www.dongqiudi.com/team/50000654.html
图尔库国际
https://www.dongqiudi.com/team/50000667.html
古比斯
https://www.dongqiudi.com/team/50000675.html
汉卡
https://www.dongqiudi.com/team/50002103.html
埃尔维斯
https://www.dongqiudi.com/team/50000663.html
拉赫蒂
https://www.dongqiudi.com/team/50006998.html
塞伊奈约基
https://www.dongqiudi.com/team/50002139.html
HIFK赫尔辛基
https://www.dongqiudi.com/team/50000672.html
马里汉姆
https://www.dongqiudi.com/team/50000655.html
哈卡
https://www.dongqiudi.com/team/50000665.html
TPS图尔库
https://www.dongqiudi.com/team/50000658.html
罗瓦涅米
https://www.dongqiudi.com/data/54
https://www.dongqiudi.com/team/50000457.html
布隆德比
https://www.dongqiudi.com/team/50000461.html
奥胡斯
https://www.dongqiudi.com/team/50000459.html
中日德兰
https

https://www.dongqiudi.com/team/50001434.html
什切青波贡
https://www.dongqiudi.com/team/50002617.html
弗罗茨瓦夫
https://www.dongqiudi.com/team/50001428.html
扎布热矿工
https://www.dongqiudi.com/team/50001436.html
卢宾盆地
https://www.dongqiudi.com/team/50001440.html
乔治罗尼亚
https://www.dongqiudi.com/team/50002629.html
格但斯克莱吉亚
https://www.dongqiudi.com/team/50001426.html
波兹南莱赫
https://www.dongqiudi.com/team/50001424.html
普沃茨克维斯瓦
https://www.dongqiudi.com/team/50001437.html
克拉科维亚
https://www.dongqiudi.com/team/50001443.html
皮亚斯特
https://www.dongqiudi.com/team/50001420.html
克拉科夫维斯瓦
https://www.dongqiudi.com/team/50002622.html
梅莱茨钢铁
https://www.dongqiudi.com/team/50002623.html
波兹南瓦塔
https://www.dongqiudi.com/team/50001438.html
波德比斯基
https://www.dongqiudi.com/data/67
https://www.dongqiudi.com/team/50002843.html
开罗国民
https://www.dongqiudi.com/team/50074478.html
克娄巴特拉
https://www.dongqiudi.com/team/50008207.html
阿斯旺
https://www.dongqiudi.com/team/50002853.html
扎马雷克体育
https://www.dongqiudi.com/team/50002850.html
亚

https://www.dongqiudi.com/team/50003968.html
保克什
https://www.dongqiudi.com/team/50000925.html
沙勒格斯基
https://www.dongqiudi.com/team/50006257.html
迈泽克韦什德
https://www.dongqiudi.com/team/50000922.html
新佩斯
https://www.dongqiudi.com/team/50051134.html
布达弗基
https://www.dongqiudi.com/team/50000929.html
布达佩斯捍卫者
https://www.dongqiudi.com/team/50000932.html
迪欧斯捷尔
https://www.dongqiudi.com/data/81
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/82
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/83
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/84
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/85
https://www.dongqiudi.com/team/50001906.html
伯尔尼年轻人
https://www.dongqiudi.com/team/50001905.html
巴塞尔
https://www.dongqiudi.com/team/50001908.html
圣加仑
https://www.dongqiudi.com/tea

https://www.dongqiudi.com/team/50000360.html
圣菲独立
https://www.dongqiudi.com/team/50013382.html
科尔多瓦美洲虎
https://www.dongqiudi.com/team/50000349.html
巴兰基亚青年
https://www.dongqiudi.com/team/50004993.html
股权保险体育
https://www.dongqiudi.com/team/50000356.html
百万富翁
https://www.dongqiudi.com/team/50000348.html
卡尔达斯十一人
https://www.dongqiudi.com/team/50005001.html
博亚卡爱国者
https://www.dongqiudi.com/team/50004995.html
里奥内格罗老鹰
https://www.dongqiudi.com/data/102
https://www.dongqiudi.com/team/50000320.html
天主大学
https://www.dongqiudi.com/team/50004896.html
拉卡莱拉联合
https://www.dongqiudi.com/team/50000315.html
西班牙联盟
https://www.dongqiudi.com/team/50004894.html
库里科联合
https://www.dongqiudi.com/team/50004736.html
安托法加斯塔
https://www.dongqiudi.com/team/50000314.html
智利大学
https://www.dongqiudi.com/team/50000311.html
瓦奇巴托
https://www.dongqiudi.com/team/50000324.html
巴勒斯坦人
https://www.dongqiudi.com/team/50000325.html
科布雷萨尔
https://www.dongqiudi.com/team/50000310.html
康塞普西翁大学
https://www.dongqiudi.com/team/50000309

https://www.dongqiudi.com/team/50000948.html
阿克拉内斯竞技
https://www.dongqiudi.com/team/50002200.html
科帕沃于尔
https://www.dongqiudi.com/team/50000951.html
维京古
https://www.dongqiudi.com/team/50006875.html
格罗塔
https://www.dongqiudi.com/team/50002577.html
弗约尼尔
https://www.dongqiudi.com/data/121
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/122
https://www.dongqiudi.com/team/50003687.html
纽波特
https://www.dongqiudi.com/team/50000620.html
绿色森林流浪
https://www.dongqiudi.com/team/50000605.html
卡莱尔联
https://www.dongqiudi.com/team/50000593.html
切尔滕汉姆
https://www.dongqiudi.com/team/50000611.html
莫克姆
https://www.dongqiudi.com/team/50000610.html
埃克塞特城
https://www.dongqiudi.com/team/50007707.html
索尔福德城
https://www.dongqiudi.com/team/50000557.html
沃尔索尔
https://www.dongqiudi.com/team/50000596.html
剑桥联
https://www.dongqiudi.com/team/50000567.html
科尔切斯特联
https://www.dongqiudi.com/team/50002432.html
克劳利
https://www.dongqiudi.com/team/50000597.html
莱顿东方
h

...

***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/209
https://www.dongqiudi.com/team/50001877.html
哈尔姆斯塔德
https://www.dongqiudi.com/team/50002117.html
迪格弗斯
https://www.dongqiudi.com/team/50004728.html
延雪平南区
https://www.dongqiudi.com/team/50001897.html
奥斯达
https://www.dongqiudi.com/team/50005501.html
阿卡波利斯
https://www.dongqiudi.com/team/50001889.html
松兹瓦尔
https://www.dongqiudi.com/team/50001896.html
瓦斯特拉斯
https://www.dongqiudi.com/team/50001892.html
布莱格
https://www.dongqiudi.com/team/50003142.html
埃斯基尔斯蒂纳竞技
https://www.dongqiudi.com/team/50001900.html
盖斯
https://www.dongqiudi.com/team/50002174.html
诺尔比
https://www.dongqiudi.com/team/50001883.html
奥基迪
https://www.dongqiudi.com/team/50001884.html
特雷勒堡
https://www.dongqiudi.com/team/50008379.html
达库德
https://www.dongqiudi.com/team/50002115.html
于默奥
https://www.dongqiudi.com/team/50002114.html
卢恩斯基尔
https://www.dongqiudi.com/data/210
***************************
      非有效页面          
***

https://www.dongqiudi.com/team/50004493.html
中国台北女足
https://www.dongqiudi.com/data/227
https://www.dongqiudi.com/team/50009238.html
西贡
https://www.dongqiudi.com/team/50014468.html
越电信
https://www.dongqiudi.com/team/50008412.html
胡志明市
https://www.dongqiudi.com/team/50002990.html
嘉莱黄英
https://www.dongqiudi.com/team/50002986.html
平阳
https://www.dongqiudi.com/team/50007052.html
广宁
https://www.dongqiudi.com/team/50007053.html
河内
https://www.dongqiudi.com/team/50007007.html
清化
https://www.dongqiudi.com/team/50002987.html
岘港
https://www.dongqiudi.com/team/50002995.html
乂安蓝江
https://www.dongqiudi.com/team/50070468.html
河静红林
https://www.dongqiudi.com/team/50004742.html
南定
https://www.dongqiudi.com/team/50002991.html
海防
https://www.dongqiudi.com/team/50008378.html
广南
https://www.dongqiudi.com/data/228
https://www.dongqiudi.com/data/229
https://www.dongqiudi.com/team/50000131.html
索利戈尔斯克矿工
https://www.dongqiudi.com/team/50000125.html
鲍里索夫
https://www.dongqiudi.com/team/50000130.html
格罗德诺涅曼
https:

https://www.dongqiudi.com/data/286
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/287
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/288
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/289
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/290
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/291
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/292
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/293
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/294
***************************
      非有效页面          
***********************
https://www.dongqiu

***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/362
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/363
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/364
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/365
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/366
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/367
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/368
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/369
***************************
      非有效页面          
***********************
https://www.dongqiudi.com/data/370
*******************

KeyboardInterrupt: 

### 爬取球员信息

In [2]:
def parse_player_html(html, url):
    player_infos = {}
    player_infos["url"] = url
    soup = BeautifulSoup(html, 'lxml')
    player_content = soup.find(name="div", class_="player-con")
    info_left = player_content.find(name="div", class_="info-left")

    #解析基本信息
    base_info = {}
    for item in info_left.find_all(name='li'):
        children = item.contents
        if len(children) == 2:
            base_info[children[0].string.rstrip("：").replace(" ", "")] = children[1]
        else:
            base_info[children[0].string.rstrip("：").replace(" ", "")] = ""

    china_name = info_left.find(name="p", class_="china-name")
    if china_name:
        base_info["中文名"] = china_name.contents[0]

    english_name = info_left.find(name="p", class_="en-name")
    if english_name:
        base_info["英文名"] = english_name.string
    player_infos["base_info"] = base_info

    #解析比赛信息
    list_ = []
    player_left = player_content.find(name='div', class_='player-left')
    player_data = player_left.find(name="div", class_="player-data")
    match_data = player_data.find(name="div", class_="match-data")
    match_data_con = match_data.find(name='div', class_="match-data-con")
    try:
        table = match_data_con.div.div
        for row in table.find_all(name="p"):
        #     print(row)
            l = []
            for item in row.find_all(name="span"):
                l.append(item.string)
            list_.append(l)
    except:
        pass
    player_infos["match_data"] = list_

    #解析荣誉信息
    honor_data = player_data.find(name="div", class_="hornor-record")
    hd = []
    try:
        for item in honor_data.find_all(name='div', class_="hornor-list"):
        #     print(item.prettify())
            l = []
            l.append(item.find(name="span", class_="champion").string)
            l.append(item.find(name="b").string)
            hd.append(l)
    except:
        pass
    player_infos["honor_data"] = hd

    #解析转会信息
    transfer_data = player_data.find(name='div', class_="transfer-data")
    trans_table = []
    try:
        for item in transfer_data.find_all(name="div", class_="transfer"):
        #     print(item.prettify())
            l = []
            date = item.find(name="span", class_="date").string
            l.append(date)
            old = item.find(name="span", class_="team-icon team-a").b.string
            l.append(old)
            new = item.find(name="span", class_="team-icon team-b").b.string
            l.append(new)
            trans_table.append(l)
    except:
        pass
    player_infos["transfer_data"]=trans_table

    #解析受伤数据
    injured = player_data.find(name="div", class_="injured")
    injured_table = []
    try:
        for item in injured.find_all(name="p", class_="injured-item"):
            # print(item)
            row = []
            for i in item.find_all(name="span"):
                row.append(i.string)
            injured_table.append(row)
    except:
        pass
    player_infos["injured_data"] = injured_table

    #解析能力数据
    cap_dic = {}
    player_right = player_content.find(name="div", class_="player-right")
    capability = player_right.find(name="div", class_="capability")
    if capability:
        average = capability.find(name="p", class_="average")
        box_chart = capability.find(name='div', class_="box_chart")
        aver = average.contents
        cap_dic[aver[0]] = aver[1].string
        first = True
        for item in box_chart.find_all(name="div"):
            if first:
                first = False
            else:
                ite = item.contents
                cap_dic[ite[0]] = ite[1].string
    player_infos["capability_data"] = cap_dic
#     print(player_infos["base_info"]["中文名"])
    return player_infos


In [3]:
def get_players_data(team_url):
    driver = webdriver.Chrome()
    try:
        driver.get(team_url) 
        #         找到每一个球员的条目
        items = driver.find_elements_by_class_name("analysis-list-item")
        for item in items:
#             模拟点击
            item.click()
            if len(driver.window_handles) > 1:#如果item可以点击，则窗口数肯定大于1
                driver.switch_to.window(driver.window_handles[-1])#切换窗口
#                 print(driver.window_handles)
                print(driver.current_url)
                player_info = parse_player_html(driver.page_source, driver.current_url)
                with open("playersData_1100-1200.txt", "a+", encoding="utf-8") as f:
                    info = "{}\n".format(player_info)
                    f.write(info)
                driver.close()#关闭页面
                driver.switch_to.window(driver.window_handles[0])#切换回原来的主页面
#             else:
#                 print("else")
    except NoSuchElementException:
        print("----------------------\n            找不到该元素          \n----------------")
    finally:
        driver.quit()#关闭浏览器

In [6]:
with open("teamsUrl.txt", "r", encoding="utf-8") as f:
    team_count = 0
    for team_url in f.readlines():
        print("-----------------------------------")
        print(team_url)
        team_count += 1
        print(team_count)
        if team_count >= 1145 and team_count < 1170:
            get_players_data(team_url)

-----------------------------------
https://www.dongqiudi.com/team/50000516.html

1
-----------------------------------
https://www.dongqiudi.com/team/50000535.html

2
-----------------------------------
https://www.dongqiudi.com/team/50000515.html

3
-----------------------------------
https://www.dongqiudi.com/team/50000527.html

4
-----------------------------------
https://www.dongqiudi.com/team/50000528.html

5
-----------------------------------
https://www.dongqiudi.com/team/50000523.html

6
-----------------------------------
https://www.dongqiudi.com/team/50000529.html

7
-----------------------------------
https://www.dongqiudi.com/team/50000514.html

8
-----------------------------------
https://www.dongqiudi.com/team/50000518.html

9
-----------------------------------
https://www.dongqiudi.com/team/50000537.html

10
-----------------------------------
https://www.dongqiudi.com/team/50000533.html

11
-----------------------------------
https://www.dongqiudi.com/team/5000051

https://www.dongqiudi.com/team/50007140.html

636
-----------------------------------
https://www.dongqiudi.com/team/50005719.html

637
-----------------------------------
https://www.dongqiudi.com/team/50007193.html

638
-----------------------------------
https://www.dongqiudi.com/team/50005446.html

639
-----------------------------------
https://www.dongqiudi.com/team/50005445.html

640
-----------------------------------
https://www.dongqiudi.com/team/50005439.html

641
-----------------------------------
https://www.dongqiudi.com/team/50005441.html

642
-----------------------------------
https://www.dongqiudi.com/team/50004602.html

643
-----------------------------------
https://www.dongqiudi.com/team/50005438.html

644
-----------------------------------
https://www.dongqiudi.com/team/50072222.html

645
-----------------------------------
https://www.dongqiudi.com/team/50005442.html

646
-----------------------------------
https://www.dongqiudi.com/team/50005443.html

647
----

https://www.dongqiudi.com/team/50005178.html

1124
-----------------------------------
https://www.dongqiudi.com/team/50001313.html

1125
-----------------------------------
https://www.dongqiudi.com/team/50001327.html

1126
-----------------------------------
https://www.dongqiudi.com/team/50023466.html

1127
-----------------------------------
https://www.dongqiudi.com/team/50023545.html

1128
-----------------------------------
https://www.dongqiudi.com/team/50022681.html

1129
-----------------------------------
https://www.dongqiudi.com/team/50023548.html

1130
-----------------------------------
https://www.dongqiudi.com/team/50022674.html

1131
-----------------------------------
https://www.dongqiudi.com/team/50039184.html

1132
-----------------------------------
https://www.dongqiudi.com/team/50079397.html

1133
-----------------------------------
https://www.dongqiudi.com/team/50024691.html

1134
-----------------------------------
https://www.dongqiudi.com/team/50022679.htm

https://www.dongqiudi.com/player/50119255.html
https://www.dongqiudi.com/player/50554517.html
https://www.dongqiudi.com/player/50204586.html
-----------------------------------
https://www.dongqiudi.com/team/50001884.html

1149
https://www.dongqiudi.com/player/50237173.html
https://www.dongqiudi.com/player/50022283.html
https://www.dongqiudi.com/player/50240784.html
https://www.dongqiudi.com/player/50081892.html
https://www.dongqiudi.com/player/50506613.html
https://www.dongqiudi.com/player/50577082.html
https://www.dongqiudi.com/player/50176703.html
https://www.dongqiudi.com/player/50274070.html
https://www.dongqiudi.com/player/50184667.html
https://www.dongqiudi.com/player/50165300.html
https://www.dongqiudi.com/player/50226767.html
https://www.dongqiudi.com/player/50570242.html
https://www.dongqiudi.com/player/50570241.html
https://www.dongqiudi.com/player/50495715.html
https://www.dongqiudi.com/player/50259959.html
https://www.dongqiudi.com/player/50054675.html
https://www.dongqiud

https://www.dongqiudi.com/player/50508244.html
https://www.dongqiudi.com/player/50417322.html
https://www.dongqiudi.com/player/50035817.html
https://www.dongqiudi.com/player/50259229.html
https://www.dongqiudi.com/player/50013878.html
https://www.dongqiudi.com/player/50508245.html
https://www.dongqiudi.com/player/50181381.html
https://www.dongqiudi.com/player/50035571.html
https://www.dongqiudi.com/player/50588742.html
https://www.dongqiudi.com/player/50227685.html
-----------------------------------
https://www.dongqiudi.com/team/50000064.html

1155
https://www.dongqiudi.com/player/50490262.html
https://www.dongqiudi.com/player/50254778.html
https://www.dongqiudi.com/player/50234322.html
https://www.dongqiudi.com/player/50151673.html
https://www.dongqiudi.com/player/50452928.html
https://www.dongqiudi.com/player/50100731.html
https://www.dongqiudi.com/player/50219311.html
https://www.dongqiudi.com/player/50184582.html
https://www.dongqiudi.com/player/50092151.html
https://www.dongqiud

https://www.dongqiudi.com/player/50433758.html
https://www.dongqiudi.com/player/50404892.html
https://www.dongqiudi.com/player/50262594.html
https://www.dongqiudi.com/player/50123026.html
https://www.dongqiudi.com/player/50588780.html
https://www.dongqiudi.com/player/50261287.html
https://www.dongqiudi.com/player/50588777.html
https://www.dongqiudi.com/player/50145406.html
https://www.dongqiudi.com/player/50499058.html
https://www.dongqiudi.com/player/50588775.html
https://www.dongqiudi.com/player/50203841.html
https://www.dongqiudi.com/player/50410773.html
-----------------------------------
https://www.dongqiudi.com/team/50005054.html

1161
https://www.dongqiudi.com/player/50587994.html
https://www.dongqiudi.com/player/50496338.html
https://www.dongqiudi.com/player/50172748.html
https://www.dongqiudi.com/player/50234826.html
https://www.dongqiudi.com/player/50577603.html
https://www.dongqiudi.com/player/50475859.html
https://www.dongqiudi.com/player/50018724.html
https://www.dongqiud

https://www.dongqiudi.com/player/50255862.html
https://www.dongqiudi.com/player/50451218.html
https://www.dongqiudi.com/player/50207973.html
https://www.dongqiudi.com/player/50113959.html
https://www.dongqiudi.com/player/50588764.html
https://www.dongqiudi.com/player/50010537.html
https://www.dongqiudi.com/player/50588765.html
https://www.dongqiudi.com/player/50109242.html
https://www.dongqiudi.com/player/50509870.html
https://www.dongqiudi.com/player/50259694.html
https://www.dongqiudi.com/player/50092548.html
https://www.dongqiudi.com/player/50254783.html
https://www.dongqiudi.com/player/50011482.html
https://www.dongqiudi.com/player/50010904.html
https://www.dongqiudi.com/player/50013675.html
https://www.dongqiudi.com/player/50237286.html
https://www.dongqiudi.com/player/50137877.html
https://www.dongqiudi.com/player/50147020.html
https://www.dongqiudi.com/player/50066510.html
https://www.dongqiudi.com/player/50011682.html
https://www.dongqiudi.com/player/50517874.html
https://www.d